<a href="https://colab.research.google.com/github/Blantranderozari/Blantranderozari/blob/main/HOML_10_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Image Classifier using Sequentials API

Taken from Hands-on Machine Learning with Scikit-Learn and Tensorflow